In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Data Aggregation") \
    .getOrCreate()

In [2]:
listings = spark.read.csv("data/listings.csv.gz", 
    header=True,
    inferSchema=True,
    sep=",",
    quote='"',
    escape='"',
    multiLine=True,
    mode="PERMISSIVE"
)

In [11]:
for field in listings.schema:
    print(field)

StructField('id', LongType(), True)
StructField('listing_url', StringType(), True)
StructField('scrape_id', LongType(), True)
StructField('last_scraped', DateType(), True)
StructField('source', StringType(), True)
StructField('name', StringType(), True)
StructField('description', StringType(), True)
StructField('neighborhood_overview', StringType(), True)
StructField('picture_url', StringType(), True)
StructField('host_id', IntegerType(), True)
StructField('host_url', StringType(), True)
StructField('host_name', StringType(), True)
StructField('host_since', DateType(), True)
StructField('host_location', StringType(), True)
StructField('host_about', StringType(), True)
StructField('host_response_time', StringType(), True)
StructField('host_response_rate', StringType(), True)
StructField('host_acceptance_rate', StringType(), True)
StructField('host_is_superhost', StringType(), True)
StructField('host_thumbnail_url', StringType(), True)
StructField('host_picture_url', StringType(), True)


In [3]:
listings \
  .groupby(listings.property_type) \
  .count() \
  .show(truncate=False)

+----------------------------------+-----+
|property_type                     |count|
+----------------------------------+-----+
|Private room in lighthouse        |2    |
|Private room in loft              |154  |
|Private room in earthen home      |2    |
|Entire chalet                     |5    |
|Earthen home                      |1    |
|Farm stay                         |4    |
|Entire rental unit                |40799|
|Shared room in hostel             |61   |
|Shared room                       |1    |
|Private room in condo             |3255 |
|Room in boutique hotel            |229  |
|Private room in religious building|4    |
|Room in bed and breakfast         |18   |
|Private room in casa particular   |62   |
|Private room in bungalow          |63   |
|Entire cabin                      |50   |
|Entire guesthouse                 |226  |
|Hut                               |4    |
|Private room in nature lodge      |4    |
|Entire guest suite                |175  |
+----------

In [4]:
import pyspark.sql.functions as F

listings \
  .groupby(listings.property_type) \
  .agg(
    F.count('property_type').alias('count')
  ) \
  .orderBy('count', ascending=[False]) \
  .show(truncate=False)

+----------------------------------+-----+
|property_type                     |count|
+----------------------------------+-----+
|Entire rental unit                |40799|
|Private room in rental unit       |14573|
|Private room in home              |11826|
|Entire home                       |8938 |
|Entire condo                      |8438 |
|Private room in condo             |3255 |
|Entire serviced apartment         |1840 |
|Private room in townhouse         |1189 |
|Entire townhouse                  |1069 |
|Room in hotel                     |1042 |
|Private room in bed and breakfast |495  |
|Private room in guesthouse        |371  |
|Entire loft                       |339  |
|Room in boutique hotel            |229  |
|Entire guesthouse                 |226  |
|Entire guest suite                |175  |
|Private room in guest suite       |170  |
|Private room in loft              |154  |
|Private room in serviced apartment|154  |
|Private room                      |103  |
+----------

In [6]:

listings \
  .groupby(listings.property_type) \
  .agg(
    F.count('property_type').alias('count'),
    F.avg('review_scores_location')
  ) \
  .orderBy('count', ascending=[False]) \
  .show(truncate=False)

+----------------------------------+-----+---------------------------+
|property_type                     |count|avg(review_scores_location)|
+----------------------------------+-----+---------------------------+
|Entire rental unit                |39372|4.72794867465818           |
|Private room in rental unit       |14837|4.732797964376552          |
|Private room in home              |11835|4.6990531037827195         |
|Entire condo                      |8800 |4.763906656266228          |
|Entire home                       |8756 |4.722269211782102          |
|Private room in condo             |3515 |4.7656638325703            |
|Entire serviced apartment         |1852 |4.7218975332068345         |
|Private room in townhouse         |1294 |4.75918656056587           |
|Entire townhouse                  |1123 |4.812793814432991          |
|Room in hotel                     |783  |4.649164619164618          |
|Private room in bed and breakfast |486  |4.7173976608187145         |
|Entir

In [5]:
reviews = spark.read.csv("data/reviews.csv.gz", 
    header=True,
    inferSchema=True,
    sep=",",
    quote='"',
    escape='"',
    multiLine=True,
    mode="PERMISSIVE"
)

In [6]:
for field in reviews.schema:
    print(field)

StructField('listing_id', LongType(), True)
StructField('id', LongType(), True)
StructField('date', DateType(), True)
StructField('reviewer_id', IntegerType(), True)
StructField('reviewer_name', StringType(), True)
StructField('comments', StringType(), True)


In [9]:
listings_reviews = listings.join(
    reviews, listings.id == reviews.listing_id, how='inner'
)

In [12]:
listings_reviews \
  .groupBy('id') \
  .agg(
    F.count('id').alias('num_reviews')
  ) \
  .show()

+----------+-----------+
|listing_id|num_reviews|
+----------+-----------+
|     78606|          2|
|    444886|         12|
|    466017|         28|
|   2736493|          4|
|   2557853|         89|
|   3132302|          3|
|   3917692|          1|
|   3734796|          5|
|   3997029|          7|
|   4361078|         70|
|   5520243|          5|
|   5355817|         18|
|   6311069|          3|
|   5921026|         40|
|   6552071|          1|
|   6606418|        252|
|   6651481|         48|
|   7188835|        206|
|   7709953|         34|
|   7943188|         70|
+----------+-----------+
only showing top 20 rows


In [14]:
reviews_per_listing = listings_reviews \
  .groupBy(listings.id, listings.name) \
  .agg(
    F.count(reviews.id).alias('num_reviews')
  ) \
  .orderBy('num_reviews', ascending=False) \
  .show(truncate=False)

+--------+--------------------------------------------------+-----------+
|id      |name                                              |num_reviews|
+--------+--------------------------------------------------+-----------+
|47408549|Double Room+ Ensuite                              |1855       |
|30760930|Double Garden View room - London House Hotel***   |1682       |
|43120947|Private double room with en suite facilities      |1615       |
|19670926|Locke Studio Apartment at Leman Locke             |1436       |
|45006692|Budget Double Room In Colliers Hotel.             |1433       |
|1436172 |Cosy Double in Kings Cross Houseshare nr Eurostar |1195       |
|2126708 |London's best transport hub 5 mins walk! Safe too!|1122       |
|1436177 |En-suite Double in Kings Cross Houseshare Eurostar|1005       |
|47438714|KX Basic- Small Double- shared bathroom           |978        |
|3855375 |Double in Kings Cross Houseshare nr Eurostar      |973        |
|46233904|Superior Studio, avg size 23